In [1]:
# import package
import os
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
import math
import sys
import random
import csv
 
# mount colab on google drive
from google.colab import drive
drive.mount('/content/drive')
 
# change dir, root dir
os.chdir('/content')
 
# change dir, file dir
os.chdir('drive/My Drive/codes/socomp')

Mounted at /content/drive


In [2]:
# read GroupTopic.txt
gt_lines = []
with open('GroupTopic.txt') as gt_f:
    gt_lines = gt_f.readlines()
gt_f.close()

group_id = []
group_organizer = []
group_attribute = []
gt_count = 1
for gt_line in gt_lines:
    if gt_count % 2 == 1:
      gt_id, gt_organizer = gt_line.split()
      group_id.append(int(gt_id[1:]))
      if gt_organizer == 'null':
        group_organizer.append(int(-1))
      else:
        group_organizer.append(int(gt_organizer[1:]))
    else:
      read_one_group_attribute = gt_line.split()
      num_group_attribute_list = []
      for i in range(len(read_one_group_attribute)):
        num_group_attribute_list.append(int(read_one_group_attribute[i][1:]))
      group_attribute.append(num_group_attribute_list)
    gt_count += 1

gt_data = zip(group_id, group_organizer, group_attribute)
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_organizer', 'group_attribute'])
gt_df = gt_df[gt_df['group_organizer'] != -1]
gt_df = gt_df.drop('group_organizer', 1)
max_group_id = gt_df['group_id'].max()
group_amount = len(gt_df.index)
gt_df

,group_id,group_attribute
0,0,"[85, 89, 124, 234, 738, 2527, 3007, 3135, 5915..."
1,1,"[212, 512, 3554]"
2,2,[1566]
3,3,"[244, 309, 1322, 1502, 3336, 4422, 8649, 9816,..."
4,4,"[638, 9753, 9772, 15067, 15984, 18874, 19061]"
...,...,...
477,477,"[1429, 7929, 18340, 29702]"
478,478,"[484, 1307, 2064, 3880, 6364, 10117, 15236, 15..."
479,479,"[9772, 10002, 10024, 10413]"
480,480,"[1237, 1273, 1543, 10050, 10080, 10232, 17665,..."


In [3]:
# read MemberTopic.txt
mt_lines = []
with open('MemberTopic.txt') as mt_f:
    mt_lines = mt_f.readlines()
mt_f.close()

member_id = []
member_attribute = []
mt_count = 1
for mt_line in mt_lines:
    if mt_count % 2 == 1:
      member_id.append(int(mt_line[1:-1]))
    else:
      read_one_member_attribute = mt_line.split()
      num_member_attribute_list = []
      for i in range(len(read_one_member_attribute)):
        num_member_attribute_list.append(int(read_one_member_attribute[i][1:]))
      member_attribute.append(num_member_attribute_list)
    mt_count += 1

mt_data = zip(member_id, member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df = mt_df[mt_df.astype(str)['member_attribute'] != '[]']
max_member_id = mt_df['member_id'].max()
member_amount = len(mt_df.index)
mt_df

,member_id,member_attribute
0,0,"[10485, 498]"
1,1,"[1201, 914, 933, 638, 590, 726, 242, 1724, 158..."
2,2,"[10495, 9753, 491, 1201, 638, 2926, 15892, 229..."
3,3,"[11660, 11046, 2151]"
4,4,"[1201, 933, 230, 2899, 1755, 3282, 1923, 228, ..."
...,...,...
73677,73677,[10050]
73680,73680,"[1155, 17195, 60, 17866, 10232, 17030, 713, 24..."
73681,73681,"[240, 3336, 495, 22440, 29431, 16551, 4422, 24..."
73682,73682,"[6761, 6490, 1526, 1307, 1330, 2088, 2064, 445..."


In [4]:
# read Group-Event folder
group_event_id = []
event_id = []
event_limit = []
event_time = []
event_organizer = []
yes_id = []
no_id = []
maybe_id = []
root = "./GroupEvent"
for i in range(0, 482):
  file = "G" + str(i) + ".txt"
  full_path = os.path.join(root, file)
  ge_group_id = file[:-4]
  with open(full_path) as ge_f:
    ge_lines = ge_f.readlines()
  ge_f.close()
  ge_count = 0
  for ge_line in ge_lines:
    ge_count += 1
    if ge_count % 5 == 0:
      maybe_id.append(ge_line.split())
    elif ge_count % 4 == 0:
      read_one_no_id = ge_line.split()
      num_no_id_list = []
      for i in range(len(read_one_no_id)):
        if read_one_no_id[i] != 'null':
          num_no_id_list.append(int(read_one_no_id[i][1:]))
      no_id.append(num_no_id_list)
    elif ge_count % 3 == 0:
      read_one_yes_id = ge_line.split()
      num_yes_id_list = []
      for i in range(len(read_one_yes_id)):
        if read_one_yes_id[i] != 'null':
          num_yes_id_list.append(int(read_one_yes_id[i][1:]))
      yes_id.append(num_yes_id_list)
    elif ge_count % 2 == 0:
      event_organizer.append(ge_line.split())
    else:
      ge_event_id, ge_event_limit, ge_event_time = ge_line.split()
      event_id.append(int(ge_event_id[1:]))
      event_limit.append(ge_event_limit)
      event_time.append(ge_event_time)
      group_event_id.append(int(ge_group_id[1:]))
    if ge_count == 5:
      ge_count = 0

ge_data = zip(group_event_id, event_id, event_limit, event_time, event_organizer, yes_id, no_id, maybe_id)
ge_df = pd.DataFrame(ge_data, columns = ['group_id', 'event_id', 'event_limit', 'event_time', 'event_organizer', 'yes_id', 'no_id', 'maybe_id'])
ge_df[['event_limit', 'event_time']] = ge_df[['event_limit', 'event_time']].astype(int)

# remove column maybe_id
ge_df = ge_df.drop('maybe_id', 1)

# remove useless  columns 
ge_df = ge_df.drop('event_organizer', 1)
ge_df = ge_df.drop('event_limit', 1)
ge_df = ge_df.drop('no_id', 1)
ge_df = ge_df.drop('event_time', 1)
ge_df = ge_df[ge_df.astype(str)['yes_id'] != '[]']
max_event_id = ge_df['event_id'].max()
event_amount = len(ge_df.index) 
ge_df

,group_id,event_id,yes_id
0,0,932,[48462]
1,0,947,"[48462, 56494, 62049, 70873, 48571]"
2,0,1742,[48462]
3,0,3372,"[48462, 55059, 56494, 70873]"
4,0,3373,"[48462, 21224, 27288, 63893]"
...,...,...,...
55391,481,39040,"[24063, 30598, 53015, 54522, 55322, 55840, 565..."
55392,481,40188,"[24063, 30598, 34608, 41229, 45326, 46797, 553..."
55393,481,40672,"[30598, 38405, 45326, 52699, 55322, 56568, 622..."
55394,481,41507,"[30598, 55322, 55850]"


In [5]:
# randomly shuffle the rows of ge_df
ge_df = ge_df.sample(frac=1)

# split ge_df into 5 parts
p1, p2, p3, p4, p5 = np.array_split(ge_df, 5)

# train_set_1, test_set_1
train_frames = [p2, p3, p4, p5]
train_set = pd.concat(train_frames)
test_set = p1
train_set

,group_id,event_id,yes_id
16192,200,33426,"[5550, 28539, 1929, 25640, 25826, 35281, 36861..."
13019,171,40275,"[45908, 23683, 23965, 25886, 27484, 29573, 392..."
17861,210,34370,"[30455, 30578, 34911, 33983, 45519, 40738, 457..."
18650,216,8526,"[27588, 38413, 38501, 39413, 71856, 1025, 2361..."
12201,168,31774,"[24444, 33638]"
...,...,...,...
40791,343,26862,[25155]
475,4,52635,"[45233, 2405, 27266]"
5105,66,39713,"[64095, 21691, 47885, 65235, 45055]"
41767,347,36356,"[29865, 49180, 61173, 3492, 12228, 41352]"


In [6]:
# train_member_set
train_member_set = set()
train_amount = len(train_set.index)
for i in range(train_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  train_member_set.update(yes_id)
len(train_member_set)

52811

In [7]:
# test_member_set
test_member_set = set()
test_amount = len(test_set.index)
for i in range(test_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  test_member_set.update(yes_id)
len(test_member_set)

27008

In [8]:
# member_set
member_set = set()
member_amount = len(mt_df.index)
for i in range(member_amount):
  member_id = mt_df.iloc[i]['member_id']
  member_set.add(member_id)
len(member_set)

47289

In [9]:
# common_member_set, train & test & mt_df
train_test_common_set = list(set(train_member_set).intersection(test_member_set))
common_member_set = list(set(train_test_common_set).intersection(member_set))
common_member_set = [int(i) for i in common_member_set]
len(common_member_set)

14574

In [10]:
# update mt_df, 32s
member_id = common_member_set
member_attribute = [set() for i in range(len(member_id))]
for i in range(len(mt_df.index)):
  id = int(mt_df.iloc[i]['member_id'])
  if id in member_id:
    index =  member_id.index(id)
    attribute = set(mt_df.iloc[i]['member_attribute'])
    member_attribute[index] = attribute

mt_data = zip(member_id, member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,32768,{590}
1,1,"{914, 15892, 15514, 17700, 933, 1201, 1459, 16..."
2,32773,{10117}
3,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
4,32776,"{9349, 392, 3336, 264, 16011, 16270, 7319, 151..."
...,...,...
14569,65523,"{512, 15236, 10117, 16645, 2064, 16, 7319, 706..."
14570,65525,"{105477, 68229, 264, 15760, 15892, 16660, 6374..."
14571,65526,{254}
14572,32763,"{264, 2154, 41117, 10439}"


In [11]:
# group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = []
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  for j in range(len(one_group_attribute)):
    if one_group_attribute[j] not in group_attribute_set:
      group_attribute_set.append(one_group_attribute[j])
group_attribute_set = set(group_attribute_set)
print(len(group_attribute_set))

1132


In [12]:
# member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

10862


In [13]:
# group&member attribute
gm_attribute = group_attribute_set.intersection(member_attribute_set)
len(gm_attribute)

1082

In [14]:
# remove member_attribute (not in gm_attribute), and update mt_df
new_member_id = []
new_member_attribute = []
member_amount = len(mt_df.index)
for i in range(member_amount):
  new_member_id.append(mt_df.iloc[i]['member_id'])
  one_old_member_attribute = mt_df.iloc[i]['member_attribute']
  one_new_member_attribute = gm_attribute.intersection(one_old_member_attribute)
  new_member_attribute.append(one_new_member_attribute)

mt_data = zip(new_member_id, new_member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,32768,{590}
1,1,"{933, 230, 713, 1001, 590, 1998, 2767, 1201, 9..."
2,32773,{10117}
3,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
4,32776,"{9349, 392, 3336, 264, 16011, 16270, 7319, 439..."
...,...,...
14569,65523,"{512, 15236, 16645, 10117, 2615, 4265, 235, 15..."
14570,65525,"{68229, 264, 16660, 15892, 7319, 933, 4392, 12..."
14571,65526,{254}
14572,32763,"{264, 10439}"


In [15]:
# remove group_attribute (not in gm_attribute), and update gt_df
new_group_id = []
new_group_attribute = []
group_amount = len(gt_df.index)
for i in range(group_amount):
  new_group_id.append(gt_df.iloc[i]['group_id'])
  one_old_group_attribute = gt_df.iloc[i]['group_attribute']
  one_new_group_attribute = gm_attribute.intersection(one_old_group_attribute)
  new_group_attribute.append(one_new_group_attribute)

gt_data = zip(new_group_id, new_group_attribute)
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_attribute'])
gt_df

,group_id,group_attribute
0,0,"{3007, 738, 23842, 44612, 234, 19119, 16816, 1..."
1,1,"{512, 3554, 212}"
2,2,{1566}
3,3,"{19168, 10306, 4422, 3336, 8649, 1322, 10290, ..."
4,4,"{9772, 15984, 19061, 9753, 18874, 15067, 638}"
...,...,...
461,477,"{7929, 18340, 1429, 29702}"
462,478,"{484, 10117, 15236, 16165, 3880, 15720, 17028,..."
463,479,"{10024, 10002, 9772, 10413}"
464,480,"{10080, 17665, 10050, 17986, 1543, 32778, 3642..."


In [16]:
# re-caculate member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

1082


In [17]:
# re-caculate group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = set()
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  group_attribute_set.update(one_group_attribute)
print(len(group_attribute_set))

1082


In [18]:
# update train_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
train_set_amount = len(train_set.index)
for i in range(train_set_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = train_set['group_id']
event_id = train_set['event_id']
train_data = zip(group_id, event_id, new_yes_id)
train_set = pd.DataFrame(train_data, columns = ['group_id', 'event_id', 'yes_id'])
train_set

,group_id,event_id,yes_id
0,200,33426,"{41031, 25640, 1929, 5550, 35281, 49078, 48599..."
1,171,40275,"{64928, 23683, 56718, 42770, 47352, 27484, 25886}"
2,210,34370,"{33984, 45519, 30578, 37554, 33983}"
3,216,8526,"{1025, 27588, 38501, 38413, 71856, 39413, 2361..."
4,168,31774,"{24444, 33638}"
...,...,...,...
43243,343,26862,{25155}
43244,4,52635,"{27266, 2405}"
43245,66,39713,{64095}
43246,347,36356,"{12228, 3492, 61173}"


In [19]:
# matrix UTS， 8min 4s
member_amount = len(mt_df.index)
column_amount = 1
np_mt = mt_df.to_numpy()
UTS = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_attribute = np_mt[i][1]
  for j in range(column_amount, member_amount):
    j_attribute = np_mt[j][1]
    com_attribute = i_attribute.intersection(j_attribute)
    numer = float(len(com_attribute))
    denom = float(len(i_attribute) + len(j_attribute) - len(com_attribute))
    if denom != 0:
      UTS[i][j] = numer / denom
      UTS[j][i] = UTS[i][j]
  column_amount += 1

In [20]:
# member, even_set, 3min 7s
train_amount = len(train_set.index)
member_amount = len(mt_df.index)
index = [*range(member_amount)]
member_id = mt_df['member_id']
dictionary = dict(zip(index, member_id))
np_train = train_set.to_numpy()
member_event_set = [set() for i in range(member_amount)]
for i in range(train_amount):
  member_set = np_train[i][2]
  for j in member_set:
    member_index = -1
    for key, value in dictionary.items():
      if j == value:
        member_index = key
        break
    member_event_set[member_index].add(np_train[i][1])

me_data = zip(member_id, member_event_set)
me_set = pd.DataFrame(me_data, columns = ['member_id', 'member_event_set'])
me_set

,member_id,member_event_set
0,32768,{24451}
1,1,"{3760, 3921, 8069, 8070}"
2,32773,"{24545, 25414, 24839, 24727, 24287}"
3,65541,"{54883, 14156, 13367}"
4,32776,"{29445, 42260, 33686, 36822, 28025, 39833, 346..."
...,...,...
14569,65523,"{9146, 8347, 9695}"
14570,65525,"{27296, 54049, 31169, 43302, 54055, 54986, 714..."
14571,65526,"{44320, 44321, 33627, 2749, 34889, 40046, 1646..."
14572,32763,"{35974, 35975, 43273, 43274, 144, 18333, 11684..."


In [21]:
# matrix Sim, 11min 34s
member_amount = len(mt_df.index)
column_amount = 1
np_me = me_set.to_numpy() 
Sim = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_event = np_me[i][1]
  for j in range(column_amount, member_amount):
    j_event = np_me[j][1]
    com_event = i_event.intersection(j_event)
    numer = float(len(com_event))
    denom = float(len(i_event) + len(j_event) - len(com_event))
    Sim[i][j] += UTS[i][j]
    if denom != 0:
      Sim[i][j] += (numer / denom)
    Sim[j][i] = Sim[i][j]
  column_amount += 1

In [22]:
# Neighbor(a), 2min 38s, attention!!!! is a_index, not a_id
member_amount = len(mt_df.index)
index = [*range(member_amount)]
K = 10
neighbor = [set() for i in range(member_amount)]
for i in range(member_amount):
  index_Sim = zip(index, Sim[i])
  nonzero_index_Sim = [i for i in index_Sim if i[1] > 0]
  if len(nonzero_index_Sim) >= K:
    sorted_index_Sim = sorted(nonzero_index_Sim, key=lambda x: x[1], reverse=True)
    for j in range(K):
      neighbor[i].add(sorted_index_Sim[j][0])

In [23]:
# update test_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
test_set_amount = len(test_set.index)
for i in range(test_set_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = test_set['group_id']
event_id = test_set['event_id']
test_data = zip(group_id, event_id, new_yes_id)
test_set = pd.DataFrame(test_data, columns = ['group_id', 'event_id', 'yes_id'])
test_set

,group_id,event_id,yes_id
0,288,50892,{63173}
1,159,54780,"{33000, 25485}"
2,191,10981,"{65381, 47046, 4615, 5196, 1684, 4118}"
3,201,6025,"{61834, 25787, 51941, 54718}"
4,287,35466,"{26580, 21783}"
...,...,...,...
10808,328,22790,"{10913, 70723, 19593, 12587, 65137, 26066, 55257}"
10809,79,13803,{}
10810,382,49504,"{16482, 18671, 54553, 12317, 39103}"
10811,282,33284,{}


In [24]:
# test_event, test_even_attibute
group_id = []
for i in range(len(gt_df.index)):
  group_id.append(gt_df.iloc[i]['group_id'])
test_amount = len(test_set.index)
group_attribute = gt_df['group_attribute']
event_attribute = [set() for i in range(test_amount)]
even_id = test_set['event_id']
for i in range(test_amount):
  one_group_id = test_set.iloc[i]['group_id']
  if one_group_id in group_id:
    group_index = group_id.index(one_group_id)
    one_event_attribute = group_attribute[group_index]
    event_attribute[i] = one_event_attribute

test_et_data = zip(even_id, event_attribute)
test_et_set = pd.DataFrame(test_et_data, columns = ['even_id', 'event_attribute'])
test_et_set

,even_id,event_attribute
0,50892,"{444, 5981}"
1,54780,{}
2,10981,{2767}
3,6025,"{5945, 726, 590}"
4,35466,{8652}
...,...,...
10808,22790,"{1091, 4422, 16550, 17866, 491, 588, 1998, 110..."
10809,13803,"{4422, 17030, 264, 713, 1001, 17866, 15117, 83..."
10810,49504,"{256, 257, 9792, 19585, 9821, 517, 26369, 9372..."
10811,33284,"{9760, 10149, 21414, 27655, 25544, 96552, 1009..."


In [25]:
# matrix member-event, UE_test, 57s
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
index = [*range(member_amount)]
dictionary = dict(zip(index, mt_df['member_id']))
UE_test = np.zeros((member_amount, event_amount), dtype=int)
for i in range(event_amount):
  member_id = test_set.iloc[i]['yes_id']
  for j in member_id:
    for key, value in dictionary.items():
      if j == value:
        UE_test[key][i] = 1
        break

In [ ]:
# so far, so good

In [67]:
# member and member_neighbor, attribute union，22s
np_mt = mt_df.to_numpy()
member_amount = len(mt_df.index)
member_neighbor_attribute = [[] for i in range(member_amount)]
for i in range(member_amount):
  i_attribute = np_mt[i][1]
  i_neighbor = neighbor[i]
  for j in i_neighbor:
    j_attribute = np_mt[j][1]
    uni_attr = i_attribute | j_attribute
    member_neighbor_attribute[i].append(uni_attr)

In [74]:
# member R_a
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
member_R_a = [0 for i in range(member_amount)]
for i in range(member_amount):
  sum_R_a = len(UE_test[i].nonzero())
  R_a = float(sum_R_a) / event_amount
  member_R_a[i] = R_a

In [76]:
# matrix pred, 50min 21s
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
pred = np.zeros((member_amount, event_amount), dtype=float)
for i in range(member_amount):
  for j in range(event_amount):
    event_attri = event_attribute[j]
    pred_numer = 0.0
    pred_denom = 0.0
    behind_plus = 0.0
    k_index = 0
    for k in neighbor[i]:
      i_k_attri = member_neighbor_attribute[i][k_index]
      com_attri = event_attri & i_k_attri
      if len(com_attri) != 0:
        pred_numer += Sim[i][k]
      pred_denom += Sim[i][k]
      if pred_denom != 0:
        behind_plus = pred_numer / pred_denom
      k_index += 1
    pred[i][j] = member_R_a[i] + behind_plus

In [79]:
# threshold filter, 2min 29s
threshold = 0.5
for i in range(member_amount):
  for j in range(event_amount):
    if pred[i][j] > threshold:
      pred[i][j] = 1
    else:
      pred[i][j] = 0

In [83]:
# evaluate result
UE_predict = pred
TP = FP = FN = TN = Precision = Recall = Accuracy = F1_score = 0.0
for i in range(member_amount):
  for j in range(event_amount):
    if UE_predict[i][j] == 1:
      if UE_test[i][j] == 1:
        TP += 1
      else:
        FP += 1
    else:
      if UE_test[i][j] == 1:
        FN += 1
      else:
        TN += 1

if (TP+FP) != 0:
  Precision = TP / (TP+FP)
if (TP+FN) != 0:
  Recall = TP / (TP+FN)
if (TP+TN+FP+FN) != 0:
  Accuracy = (TP+TN) / (TP+TN+FP+FN)
if (Precision+Recall) != 0:
  F1_score = 2 * Precision * Recall / (Precision+Recall)

print('Precision: ' + str(Precision))
print('Recall: ' + str(Recall))
print('Accuracy: ' + str(Accuracy))
print('F1_score: ' + str(F1_score))

Precision: 0.001151786890961533
Recall: 0.7282615861296065
Accuracy: 0.8175453954929829
F1_score: 0.002299936302569768
